Thank you DigitalSreeni on YouTube for the walk-through tutorial on the following.

In this page we access our gmail that we have been subscribing to each website's investor relations email alerts with. 



In [171]:
# # "In a multipart e-mail, email.message.Message.get_payload() returns a list with one item for each part." @bnsreenu GitHub
# my_payload = email.message.Message.get_payload(msg)
# my_payload

In [172]:
# TDQM to track progress:
from tqdm.notebook import tqdm

In [173]:
# !pip install imap

In [174]:
# ! pip install yaml

In [175]:
import imaplib
import email
import yaml

from email import policy

# for bytes to string
from email.parser import BytesParser
from email.parser import Parser
from email.message import EmailMessage

In [176]:
# open saved user credentials
with open('credentials.yaml') as f:
    content = f.read()

In [177]:
# import user and pass from credentials.yaml
my_credentials = yaml.load(content, Loader=yaml.FullLoader)

In [178]:
# Load the user name and password
user, password = my_credentials["user"], my_credentials["password"]

In [179]:
# IMAP connection url
imap_url = 'imap.gmail.com'

In [180]:
# connect ro Gmail with SSL
my_mail = imaplib.IMAP4_SSL(imap_url)

In [181]:
# Log in to gmail with credentials
my_mail.login(user, password)

('OK', [b'montgash3@gmail.com authenticated (Success)'])

In [182]:
# Select inbox to fetch message
my_mail.select('Inbox')

('OK', [b'743'])

In [183]:
# define Key/Value for email filtering
# see other 
key = 'FROM'
value = 'investrel@cmsenergy.com'

In [184]:
# Get data columns(emails) matching above filitering criteria
_, data = my_mail.search(None, key, value)

In [185]:
# IDs of all emails we want based on above filtering
mail_id_list = data[0].split()
mail_id_list

[b'52', b'344', b'681']

In [186]:
# Empty list to capture messages
msgs = []


# Return whole message for all messages we have filtered
# '(RFC822)' is the command to return entire message
# 'BODY' would return just the body
for num in mail_id_list: 
    typ, data  = my_mail.fetch(num, '(RFC822)')
    msgs.append(data)

# Preview
msgs[:2]

[[(b'52 (RFC822 {10243}',
   b'Delivered-To: montgash3@gmail.com\r\nReceived: by 2002:a05:6f02:cacc:b0:61:e91d:81c with SMTP id t12csp672448rcg;\r\n        Tue, 23 Jan 2024 09:25:50 -0800 (PST)\r\nX-Google-Smtp-Source: AGHT+IHD/A2kZcsWJQbibzCMUP+qtouNU+WIOZM9PIxf+CQ4MO3zaqaVzOZeV5Y4ZwoODDx1V5HY\r\nX-Received: by 2002:ad4:5e86:0:b0:685:9ba5:4728 with SMTP id jl6-20020ad45e86000000b006859ba54728mr1766901qvb.7.1706030750103;\r\n        Tue, 23 Jan 2024 09:25:50 -0800 (PST)\r\nARC-Seal: i=1; a=rsa-sha256; t=1706030750; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=XCzD+WC1TPd4jIOmje4RXUGPEbDhivNVWLjNM3i2xzKhCIMyPjGBH0bscGxRtUNpij\r\n         TccqfZvHE7ZkzpWxgbMUIsJp7fVYHbrwGGVzP6UEm3sW08IqJxAXtiBUoOQ2iv3v3WW4\r\n         tKitqAd+N0PjtEPuwdoVXNDi7SEdCAeOM9CBO1Or/iaeyeWzC4RLkiQi5cdkX1AIF5b7\r\n         PR+4afhGzLCJAsdIsndaO0bF62s1TV4vQ+2lhEMT/+oiHEI71zRC13HVVr1e68IMx0LH\r\n         BSkZKrDSLzimnxTZp0coEBG810f//v+LFTQ81zNFFCHuDYq9MlashT5xwbdRdU7ayoef\r\n         QMAA==\r\nARC

### Now we make sense of each part of the message

In [141]:
# ! pip install bs4

In [142]:
import html
from bs4 import BeautifulSoup

In [187]:
# Loop to gather each part of the message
# Update to GitHub@bnsreenu's published edition: added '# For HTML parts'

# "In a multipart e-mail, email.message.Message.get_payload() returns a list with one item for each part." @bnsreenu GitHub
# my_payload = email.message.Message.get_payload(msg)

# my_payload

for msg in msgs[::-1]:
    for response_part in msg:
        if type(response_part) is tuple:
            my_msg=email.message_from_bytes((response_part[1]))
            print("_________________________________________")
            print ("subj:", my_msg['subject'])
            print ("from:", my_msg['from'])
            print ("body:")
            for part in my_msg.walk():  
                print(part.get_content_type())
                if part.get_content_type() == 'text/plain':
                    print (part.get_payload())
                elif part.get_content_type() == 'text/html':
                    # For HTML parts
                    html_content = part.get_payload(decode=True).decode('utf-8')
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # Extract and print all hyperlinks
                    for link in soup.find_all('a', href=True):
                        print("Hyperlink:", link['href'])


# The above is giving us: news release, all news releases, unsubscribe. In that order

_________________________________________
subj: CMS Energy Corporation - Consumers Energy and Consumers Energy
 Foundation Donate More than $11 Million in 2023
from: CMS Energy - Investor Relations <investrel@cmsenergy.com>
body:
text/html
Hyperlink: https://u11277508.ct.sendgrid.net/ls/click?upn=32r1F7qKA-2FlPf4VG4Yq9V7rIj65-2FobA5M0Y4tcQq3JTqKzMlWYDtF9q7KahnmRZk0QJ5e5I10M6q-2BkviFxa8vLoXwGmWVJ8ZJaixO-2FLaF0JIp5uAS1yeio6NuXQv61cGg1h5OdfqSzI3HqXIkAnpfSGFyaB8I8mebPPEKtE47X-2Bhcdt-2FfUoHYTGt1o1Rn9q-2BrUzjRfZ9gK2ci1chWZmuSfcqgV-2F2B75p-2BfF7c4I5jg69hM18kS291fvDgubgCnHejYJZ_cbLNfl9DC06I7yqAggRDStoEt6EUIhRidxfYOh1J-2BZf-2FJqL8Hq-2Fw5EcfayrRJbj6IHM-2B6QY1qXVC-2FumwUE-2BQq-2Bj45H42RYtbuhqtF-2BE-2FswlTISfNYwxrvQ5oXLY9Hwim5B9nU4ERot8RD7IaM0d0Elxt-2B5C4kKiRtJWlc9zJpd5uEpi8CD2JYEus8e722pCs8o1kHWbJFwIKJ-2BLa-2FCmJ8TzlSbpH7pgq7-2BaRxtCMzlDYNH02IxvE8bu-2FAh54pB-2F10JL-2FZc8iwIVpmr6t-2FfpBcHuw86S6w6NkTb3-2B8DibNnjmhfRrUFHDri3HCEFm2X34y8PWgtVIan2Kx6aXQ1pbPBlDhEhm1ECzhTm1MWg9KpLEvSX5cUXaOO7ILbpW-2F-2Bk

Ok, it's working properly. Now we have to see if it is always the case that the first link in each email sent to us will take us to the current press release. 

If NOT, then we will have to make our computer search for the correct link somehow.

In [145]:
# First we shall iterate this process with more emails in our inbox, not filtering by sender.

# Select inbox to fetch message
my_mail.select('Inbox')

# # define Key/Value for email filtering
# # see other 
# key = 'FROM'
# value = 'investrel@cmsenergy.com'

# Get data columns(emails) matching above filitering criteria
_, data = my_mail.search(None, 'ALL')


# IDs of all emails we want based on above filtering
mail_id_list = data[0].split()
mail_id_list


# Empty list to capture messages
msgs = []


# Return whole message for all messages we have filtered
# '(RFC822)' is the command to return entire message
# 'BODY' would return just the body
for num in mail_id_list: 
    typ, data  = my_mail.fetch(num, '(RFC822)')
    msgs.append(data)

In [188]:
msgs[:5]

[[(b'52 (RFC822 {10243}',
   b'Delivered-To: montgash3@gmail.com\r\nReceived: by 2002:a05:6f02:cacc:b0:61:e91d:81c with SMTP id t12csp672448rcg;\r\n        Tue, 23 Jan 2024 09:25:50 -0800 (PST)\r\nX-Google-Smtp-Source: AGHT+IHD/A2kZcsWJQbibzCMUP+qtouNU+WIOZM9PIxf+CQ4MO3zaqaVzOZeV5Y4ZwoODDx1V5HY\r\nX-Received: by 2002:ad4:5e86:0:b0:685:9ba5:4728 with SMTP id jl6-20020ad45e86000000b006859ba54728mr1766901qvb.7.1706030750103;\r\n        Tue, 23 Jan 2024 09:25:50 -0800 (PST)\r\nARC-Seal: i=1; a=rsa-sha256; t=1706030750; cv=none;\r\n        d=google.com; s=arc-20160816;\r\n        b=XCzD+WC1TPd4jIOmje4RXUGPEbDhivNVWLjNM3i2xzKhCIMyPjGBH0bscGxRtUNpij\r\n         TccqfZvHE7ZkzpWxgbMUIsJp7fVYHbrwGGVzP6UEm3sW08IqJxAXtiBUoOQ2iv3v3WW4\r\n         tKitqAd+N0PjtEPuwdoVXNDi7SEdCAeOM9CBO1Or/iaeyeWzC4RLkiQi5cdkX1AIF5b7\r\n         PR+4afhGzLCJAsdIsndaO0bF62s1TV4vQ+2lhEMT/+oiHEI71zRC13HVVr1e68IMx0LH\r\n         BSkZKrDSLzimnxTZp0coEBG810f//v+LFTQ81zNFFCHuDYq9MlashT5xwbdRdU7ayoef\r\n         QMAA==\r\nARC

In [157]:
# Loop to gather each part of the message
# Update to GitHub@bnsreenu's published edition: 
#       - added '# For HTML parts'
#       - added error handling for 'utf-8' codec.

# "In a multipart e-mail, email.message.Message.get_payload() returns a list with one item for each part." @bnsreenu GitHub
# my_payload = email.message.Message.get_payload(msg)

# my_payload

for msg in msgs[::-1]:
    for response_part in msg:
        if isinstance(response_part, tuple):
            my_msg = email.message_from_bytes(response_part[1])
            print("_________________________________________")
            print("subj:", my_msg['subject'])
            print("from:", my_msg['from'])
            print("body:")
            for part in my_msg.walk():  
                print(part.get_content_type())
                if part.get_content_type() == 'text/plain':
                    try:
                        print(part.get_payload(decode=True).decode('utf-8'))
                    except UnicodeDecodeError:
                        print(part.get_payload(decode=True).decode('latin1'))
                elif part.get_content_type() == 'text/html':
                    # For HTML parts
                    try:
                        html_content = part.get_payload(decode=True).decode('utf-8')
                    except UnicodeDecodeError:
                        html_content = part.get_payload(decode=True).decode('latin1')
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # Extract and print all hyperlinks
                    for link in soup.find_all('a', href=True):
                        print("Hyperlink:", link['href'])


# The above is giving us: news release, all news releases, unsubscribe. In that order
# IT THIS TRUE FOR ALL EMAILS??


_________________________________________
subj: Ares Commercial Real Estate Corporation: SEC Filing
from: Ares Commercial Real Estate IR Alerts <ACRE-IR-Alerts@services.eqs.com>
body:
multipart/alternative
text/plain
&nbsp;





















&nbsp;














&nbsp;















Dear Subscriber,
New SEC Filing from Ares Commercial Real Estate Corporation: 144
For more details click here 
























&nbsp;














&nbsp;
click here.



text/html
Hyperlink: https://ir.arescre.com/sec-filings/sec-documents/?filing_type%5B%5D=1&filing_type%5B%5D=2&filing_type%5B%5D=3&filing_type%5B%5D=5&filing_type%5B%5D=6&filter=Apply+Filter
_________________________________________
subj: Ares Commercial Real Estate Corporation: SEC Filing
from: Ares Commercial Real Estate IR Alerts <ACRE-IR-Alerts@services.eqs.com>
body:
multipart/alternative
text/plain
&nbsp;





















&nbsp;














&nbsp;















Dear Subscriber,
New SEC Filing from Ares Commercial Re

In [161]:
# Now instead of printing, we want to save these items into a new dataframe
import pandas as pd


In [162]:
df = pd.DataFrame

df(columns=["subj", "from", "body", "links"])

,subj,from,body,links


In [163]:
df["subj"] = my_msg['subject']

TypeError: 'type' object does not support item assignment

In [169]:
response_part_test = msgs[::-1][0]
# my_msg = email.message_from_bytes(response_part[1])

if isinstance(response_part, tuple):
            my_msg = email.message_from_bytes(response_part[1])
            print("_________________________________________")
            print("subj:", my_msg['subject'])

In [ ]:
for msg in msgs[::-1]:
    for response_part in msg:
        if isinstance(response_part, tuple):
            my_msg = email.message_from_bytes(response_part[1])
            print("_________________________________________")
            print("subj:", my_msg['subject'])
            print("from:", my_msg['from'])
            print("body:")
            for part in my_msg.walk():  
                print(part.get_content_type())
                if part.get_content_type() == 'text/plain':
                    try:
                        print(part.get_payload(decode=True).decode('utf-8'))
                    except UnicodeDecodeError:
                        print(part.get_payload(decode=True).decode('latin1'))
                elif part.get_content_type() == 'text/html':
                    # For HTML parts
                    try:
                        html_content = part.get_payload(decode=True).decode('utf-8')
                    except UnicodeDecodeError:
                        html_content = part.get_payload(decode=True).decode('latin1')
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # Extract and print all hyperlinks
                    for link in soup.find_all('a', href=True):
                        print("Hyperlink:", link['href'])
